In [1]:
import numpy as np
import pandas as pd
#import find_peaks as findpeak

In [2]:
def find_nearest(array,value):
    idx = (np.abs(array-value)).argmin()
    return idx

In [ ]:
def test_find_nearest():
    